### Introduction 

### Setup

In [ ]:
import pymongo
import pandas as pd
from pandas.io.json import json_normalize 
from pymongo import MongoClient
import numpy as np 

In [ ]:
 #point the client at mongo URI 
client = MongoClient('mongodb://localhost:27017')
#select database
db = client['tipster']

In [ ]:
form_collection = db.form
data = form_collection.find({},{"_id":0}) 
form_df = pd.DataFrame(json_normalize(data, 'form',['horse_name']))

In [ ]:
# flatten data and put into dataframe 
meetings_collection = db.meetings
data=meetings_collection.find({},{"_id":0})
meeting_runners_df= pd.DataFrame(json_normalize(data,'runners',['meeting_url','meeting_date_time']))
meeting_runners_df['won'] = np.where(meeting_runners_df['place'] == 1.0, 1,0 )
meeting_runners_df.head()

for each horse-date check if winner in any of the previous races  

for each horse get the data of the first win  
for each horse-date horse is a winner if date (strictly) > first win 

In [ ]:
winners = meeting_runners_df[meeting_runners_df.won == 1][['horse_name','meeting_date_time']]
winners.rename(columns={"meeting_date_time":"win_date"},inplace=True)
winners = winners.groupby(["horse_name"]).min().reset_index() 
X = pd.merge(meeting_runners_df,winners,on="horse_name",how="left")
X["has_winning_form"] = np.where((X.meeting_date_time > X.win_date),1,0)



generate winning_form data

#### Form against competitors
generate pairs - cartesian product of horses for each meeting
find winner.
if date of win is prior to meeting date update beaten other horse in race in 'x' to true



In [ ]:
#cartesian join get pairs of horses in the same meeting
mdf = meeting_runners_df
mdfl = pd.DataFrame(mdf[['meeting_url','horse_name','place']])
mdfr = mdfl
competitors = mdfl.merge(mdfr, on='meeting_url')
competitors['winner'] = np.where(competitors.place_x < competitors.place_y, competitors.horse_name_x, competitors.horse_name_y) 
competitors['loser'] = np.where(competitors.place_x >= competitors.place_y, competitors.horse_name_x, competitors.horse_name_y)
competitors.rename(columns={"place_x":"winner_place","place_y":"loser_place"},inplace=True)

#### collateral form   
##### if A beat B and B beat C in another race , then A has collateral form against C
##### if A beat B and A beat C and B had a better place than C against A, then B has indirect form against C 

In [ ]:
collateral_form = competitors.merge(competitors, left_on="loser", right_on="winner")
collateral_form = collateral_form[collateral_form['meeting_url_x']!=collateral_form['meeting_url_y']]

### Data|

### Features


which horses in a meeting are winners as of meeting date?  

has a horse in a meeting beaten another horse in the same meeting?

for a given pair of horses which one did better against a third horse?  
    how similar were the conditions? 

time since last race (fitness and value of form)

does the horse need to improve - is it a champion

will form improve?   
margin of victory - ( bigger implies form won't change )  (low fitness + ease => improving form) 

commentary:  
   activity during race  
   finish  
   speed 

is progressively better

above or below average time for course?

raised or lowered in class?

proven at distance 

proven for going

weight

draw - look up advantage of draw in course

type of race - handicap/flat/jumps

### Training


In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
# from sklearn.cross_validation import train_test_split 



### Predict